### load packages

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from matplotlib import cm


### load data

In [2]:
coin_df = pd.read_csv('Crypto_Price_20240108.csv')
coin_df = coin_df.drop(columns=['BSV', 'LTC'])
crix_df = pd.read_csv('new_crix.csv')
df = coin_df.merge(crix_df)
df = df.rename(columns={'price':'CRIX'})
df.index = pd.to_datetime(df['date'])
df = df.drop(columns=['date'])
df = df.apply(lambda x: np.log(x)-np.log(x.shift(1)))
df = df.dropna(how='any')
df = df[df.index >= '2017-09-06']

In [3]:
vcrix_df = pd.read_csv('vcrix.csv')
vcrix_df.index = vcrix_df['date']
vcrix_df = vcrix_df[vcrix_df.index >= '2022-01-01']


In [4]:
fig = make_subplots(rows=1, cols=1, shared_xaxes=True, vertical_spacing=0)

fig.update_layout(
    height=600,
    width=1200,
    title='',
    showlegend=False,
    font=dict(family='Times New Roman', size=14),
    plot_bgcolor='rgba(0,0,0,0)',
    paper_bgcolor='rgba(0,0,0,0)',
)

    
fig.add_trace(go.Scatter(
    y=vcrix_df['vcrix'], 
    x=vcrix_df.index,
    line=dict(color='black', width=1),
    # mode='markers', marker_size=1,  
    ), row=1, col=1)


fig.update_xaxes(
    title='',
    tickformat='%Y-%m',  
    # dtick='M3',  
    showline=True,
    linewidth=1,
    linecolor='black',
    mirror=True,
    showgrid=False,
    row=1, col=1  
)

fig.update_yaxes(
    showticklabels=True,
    title='VCRIX',
    showline=True,
    linewidth=1,
    linecolor='black',
    mirror=True,
    showgrid=False,
)

fig.show()


#### CSAD

In [5]:
def rolling_csad(coin_returns, market_returns, window):
    csad_values = []
    for i in range(len(coin_returns) - window + 1):
        window_data = coin_returns.iloc[i:i+window]
        window_market = market_returns[i:i+window].mean()
        N = len(window_data.columns)
        csad = (N / (N - 1)) * np.sum(np.abs(window_data.sub(window_market, axis=0)).mean(axis=1))
        csad_values.append(csad)
    return pd.Series(csad_values, index=coin_returns.index[window-1:])

# # E(R_{m,t})
# def moving_average(market_returns, n=30):
#     ret = np.cumsum(market_returns, dtype=float)
#     ret[n:] = ret[n:] - ret[:-n]
#     return ret[n - 1:] / n

In [6]:
window=1
rolling_csad_values = rolling_csad(df[df.columns[:-1]], df['CRIX'], window)
avg_crix = df['CRIX'].rolling(window=window).mean()
avg_crix = avg_crix.iloc[window:]
df = df.iloc[window:]


In [7]:
csad_df = pd.DataFrame()
csad_df['csad'] = rolling_csad_values
csad_df['CRIX'] = df['CRIX']
csad_df = csad_df[1:]
csad_df.index = df.index
csad_df['abs_CRIX'] = abs(df['CRIX'])
csad_df['sq_CRIX'] = np.square(df['CRIX'])
csad_df = csad_df[(csad_df.index >= '2022-01-01') & (csad_df.index <= '2023-12-31')]

In [8]:
colors = ['blue', 'red', 'green', 'orange', 'purple', 'cyan', 'magenta', 'MidnightBlue', 'brown']

fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0)

fig.update_layout(
    height=800,
    width=1000,
    title='',
    showlegend=False, 
    font=dict(family='Times New Roman', size=14),
    plot_bgcolor='rgba(0,0,0,0)',
    paper_bgcolor='rgba(0,0,0,0)',
)

for i, col in enumerate(df.columns[:-1]):
    fig.add_trace(
        go.Scatter(
            y=df[(df.index >= '2022-01-01') & (df.index <= '2023-12-31')][col], 
            x=csad_df.index,
            mode='lines',  
            name=col, 
            line=dict(width=1, color=colors[i % len(colors)]), 
             opacity=.5,
        ),
        row=1, col=1 
    )
    
fig.add_trace(go.Scatter(
    y=csad_df['csad'], 
    x=csad_df.index,
    line=dict(color='black', width=1),
    ), row=2, col=1)

# fig.add_trace(go.Scatter(
    # y=avg_crix, 
    # x=avg_crix.index,
    # line=dict(color='orange', width=1),
    # ), row=1, col=1)


fig.update_xaxes(
    title='',
    showticklabels=False,  
    showline=True,
    linewidth=1,
    linecolor='black',
    mirror=True,
    showgrid=False,
    row=1, col=1 
)

fig.update_xaxes(
    title='',
    tickformat='%Y-%m',  
    dtick='M1',  
    showline=True,
    linewidth=1,
    linecolor='black',
    mirror=True,
    showgrid=False,
    row=2, col=1
)

fig.update_yaxes(
    showticklabels=True,
    title='Log return',
    showline=True,
    linewidth=1,
    linecolor='black',
    mirror=True,
    showgrid=False,
    row=1, col=1,  
)

fig.update_yaxes(
    showticklabels=True,
    title='CSAD',
    showline=True,
    linewidth=1,
    linecolor='black',
    mirror=True,
    showgrid=False,
    row=2, col=1 
)

fig.show()


In [9]:

anime_df = pd.DataFrame()
anime_df['csad'] = csad_df['csad']
anime_df['avg_CRIX'] = avg_crix[(avg_crix.index >= '2022-01-01') & (avg_crix.index <= '2023-12-31')]


### static plot between csad and crix

In [10]:
fig = make_subplots(rows=1, cols=1, shared_xaxes=True, vertical_spacing=0)

fig.update_layout(
    height=600,
    width=600,
    title='',
    showlegend=False,  
    font=dict(family='Times New Roman', size=14),
    plot_bgcolor='rgba(0,0,0,0)',
    paper_bgcolor='rgba(0,0,0,0)',
)
    
fig.add_trace(go.Scatter(
    y=anime_df['csad'], 
    x=anime_df['avg_CRIX'],
    line=dict(color='black', width=1),
    mode='markers', marker_size=10, 
    ), row=1, col=1)



# Update x-axis for the first row
fig.update_xaxes(
    title='Expected market return',
    showticklabels=True,  # Hide tick labels
    showline=True,
    linewidth=1,
    linecolor='black',
    mirror=True,
    showgrid=False,
    row=1, col=1,
    range=[-0.2,0.2]
)


# Update y-axis for the first row
fig.update_yaxes(
    showticklabels=True,
    title='CSAD',
    showline=True,
    linewidth=1,
    linecolor='black',
    mirror=True,
    showgrid=False,
    row=1, col=1,
    range=[0, 0.15]
)

# Show the plot
fig.show()


### anime without the marker of event dates

In [ ]:

anime_df = pd.DataFrame()
anime_df['csad'] = csad_df['csad']
anime_df['avg_CRIX'] = avg_crix[(avg_crix.index >= '2022-01-01') & (avg_crix.index <= '2023-12-31')]

highlight_dates = pd.to_datetime(['2022-05-13', '2022-11-11', '2023-07-13'])

fig = make_subplots(rows=1, cols=1)

fig.update_layout(
    height=600,
    width=600,
    showlegend=False,
    font=dict(family='Times New Roman', size=14),
    plot_bgcolor='rgba(0,0,0,0)',
    paper_bgcolor='rgba(0,0,0,0)',
)

# Generate a colormap from dark to light using 'jet'
cmap = cm.get_cmap('jet', len(anime_df))
colors = [cmap(i) for i in range(len(anime_df))]
colors = ['rgba({}, {}, {}, {})'.format(int(r*255), int(g*255), int(b*255), a) for r, g, b, a in colors]

# Initial Trace
fig.add_trace(
    go.Scatter(
        x=anime_df['avg_CRIX'],
        y=anime_df['csad'],
        mode='markers',
        marker=dict(size=10, color=colors[0])
    )
)

# Configure x and y axis
fig.update_xaxes(title='Expected market return', range=[-0.2, 0.2])
fig.update_yaxes(title='CSAD', range=[0, 0.15])

# Create frames for animation
frames = [
    go.Frame(
        data=[
            go.Scatter(
                x=anime_df.iloc[:i+1]['avg_CRIX'],
                y=anime_df.iloc[:i+1]['csad'],
                mode='markers',
                marker=dict(
                    size=[20 if anime_df.index[j] in highlight_dates else 10 for j in range(i+1)],
                    color=[colors[j] if anime_df.index[j] not in highlight_dates else 'red' for j in range(i+1)],
                    symbol=['circle' if anime_df.index[j] not in highlight_dates else 'star' for j in range(i+1)]
                )
            )
        ],
        name=f"frame_{i}"
    )
    for i in range(len(anime_df))
]

fig.frames = frames

fig.update_layout(
    updatemenus=[{
        "buttons": [
            {"args": [None, {"frame": {"duration": 20, "redraw": True}, "fromcurrent": True}], "label": "Play", "method": "animate"},
            {"args": [[None], {"frame": {"duration": 0, "redraw": True}, "mode": "immediate", "transition": {"duration": 0}}], "label": "Pause", "method": "animate"}
        ],
        "direction": "left",
        "pad": {"r": 10, "t": 87},
        "showactive": False,
        "type": "buttons",
        "x": 0.1,
        "xanchor": "right",
        "y": 0,
        "yanchor": "top"
    }],
    sliders=[{
        "active": 0,
        "yanchor": "top",
        "xanchor": "left",
        "currentvalue": {
            "font": {"size": 20},
            "prefix": "Time:",
            "visible": True,
            "xanchor": "right"
        },
        "transition": {"duration": 20, "easing": "cubic-in-out"},
        "pad": {"b": 10, "t": 50},
        "len": 0.9,
        "x": 0.1,
        "y": 0,
        "steps": [{
            "args": [[f'frame_{i}'], {"frame": {"duration": 20, "redraw": True}, "mode": "immediate", "transition": {"duration": 10}}],
            "label": anime_df.index[i].strftime('%Y-%m-%d'),
            "method": "animate"
        } for i in range(len(anime_df))]
    }]
)

fig.show()

fig.write_html("animated_plot.html")

### anime with markers of the event dates

In [ ]:
anime_df = pd.DataFrame()
anime_df['csad'] = csad_df['csad']
anime_df['avg_CRIX'] = avg_crix[(avg_crix.index >= '2022-01-01') & (avg_crix.index <= '2023-12-31')]

highlight_dates = pd.to_datetime(['2022-05-13', '2022-11-11', '2023-07-13'])
highlight_labels = {
    '2022-05-13': "Terra's Crash",
    '2022-11-11': "FTX’s Bankruptcy",
    '2023-07-13': "Ripple's Win'"
}

fig = make_subplots(rows=1, cols=1)

fig.update_layout(
    height=600,
    width=600,
    showlegend=False,
    font=dict(family='Times New Roman', size=14),
    plot_bgcolor='rgba(0,0,0,0)',
    paper_bgcolor='rgba(0,0,0,0)',
)

# Generate a colormap from dark to light using 'jet'
cmap = cm.get_cmap('jet', len(anime_df))
colors = [cmap(i) for i in range(len(anime_df))]
colors = ['rgba({}, {}, {}, {})'.format(int(r*255), int(g*255), int(b*255), a) for r, g, b, a in colors]

# Initial Trace
fig.add_trace(
    go.Scatter(
        x=anime_df['avg_CRIX'],
        y=anime_df['csad'],
        mode='markers',
        marker=dict(size=10, color=colors[0])
    )
)

# Configure x and y axis
fig.update_xaxes(title='Expected market return', range=[-0.2, 0.2])
fig.update_yaxes(title='CSAD', range=[0, 0.15])

# Create frames for animation
frames = [
    go.Frame(
        data=[
            go.Scatter(
                x=anime_df.iloc[:i+1]['avg_CRIX'],
                y=anime_df.iloc[:i+1]['csad'],
                mode='markers',
                marker=dict(
                    size=[20 if anime_df.index[j] in highlight_dates else 10 for j in range(i+1)],
                    color=[colors[j] if anime_df.index[j] not in highlight_dates else 'red' for j in range(i+1)],
                    symbol=['star' if anime_df.index[j] in highlight_dates else 'circle' for j in range(i+1)]
                )
            )
        ],
        name=f"frame_{i}",
        layout=go.Layout(
            annotations=[
                dict(
                    x=anime_df.loc[date]['avg_CRIX'],
                    y=anime_df.loc[date]['csad'],
                    text=label,
                    showarrow=True,
                    arrowhead=2,
                    ax=20,
                    ay=-30
                ) for date, label in highlight_labels.items() if date in anime_df.index[:i+1]
            ]
        )
    )
    for i in range(len(anime_df))
]

fig.frames = frames

fig.update_layout(
    updatemenus=[{
        "buttons": [
            {"args": [None, {"frame": {"duration": 20, "redraw": True}, "fromcurrent": True}], "label": "Play", "method": "animate"},
            {"args": [[None], {"frame": {"duration": 0, "redraw": True}, "mode": "immediate", "transition": {"duration": 0}}], "label": "Pause", "method": "animate"}
        ],
        "direction": "left",
        "pad": {"r": 10, "t": 87},
        "showactive": False,
        "type": "buttons",
        "x": 0.1,
        "xanchor": "right",
        "y": 0,
        "yanchor": "top"
    }],
    sliders=[{
        "active": 0,
        "yanchor": "top",
        "xanchor": "left",
        "currentvalue": {
            "font": {"size": 20},
            "prefix": "Time:",
            "visible": True,
            "xanchor": "right"
        },
        "transition": {"duration": 20, "easing": "cubic-in-out"},
        "pad": {"b": 10, "t": 50},
        "len": 0.9,
        "x": 0.1,
        "y": 0,
        "steps": [{
            "args": [[f'frame_{i}'], {"frame": {"duration": 20, "redraw": True}, "mode": "immediate", "transition": {"duration": 10}}],
            "label": anime_df.index[i].strftime('%Y-%m-%d'),
            "method": "animate"
        } for i in range(len(anime_df))]
    }]
)

fig.show()

fig.write_html("animated_plot.html")


In [13]:
import statsmodels.api as sm

Y = csad_df['csad']
X = csad_df[['abs_CRIX', 'sq_CRIX']]

# Add a constant to the independent variables (X) for the intercept
X = sm.add_constant(X)

# Fit the OLS model
model = sm.OLS(Y, X).fit()

# Print the summary of the regression
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                   csad   R-squared:                       0.229
Model:                            OLS   Adj. R-squared:                  0.227
Method:                 Least Squares   F-statistic:                     108.1
Date:                Fri, 28 Jun 2024   Prob (F-statistic):           7.77e-42
Time:                        18:32:16   Log-Likelihood:                 2022.7
No. Observations:                 730   AIC:                            -4039.
Df Residuals:                     727   BIC:                            -4026.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0152      0.001     16.668      0.0